In [1]:
using CSV, DataFrames
using PyPlot

In [19]:
df = DataFrame(CSV.read("LAMOST-dr4v2-stellar.csv"));

In [20]:
sort!(df, :snrz, rev=true);

In [49]:
CSV.write("LAMOST_top_100_snr.csv", df[1:100, [:obsid]])
CSV.write("LAMOST_top_300_snr.csv", df[1:300, [:obsid]])
CSV.write("LAMOST_top_1000_snr.csv", df[1:1000, [:obsid]])

"LAMOST_top_1000_snr.csv"